In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import bibtexparser
import requests
from urllib.parse import quote

In [10]:
with open('seedPapers.bib') as bibtex_file:
  bib_database = bibtexparser.load(bibtex_file)
papers = bib_database.entries

In [6]:
desired_capabilities = DesiredCapabilities.FIREFOX
desired_capabilities["pageLoadStrategy"] = "none"

In [15]:
firefox_options = Options()
firefox_options.add_argument("--headless")
firefox_options.set_preference('permissions.default.stylesheet', 2)
firefox_options.set_preference('permissions.default.image', 2)
firefox_options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

driver = webdriver.Firefox(options=firefox_options, service=FirefoxService(GeckoDriverManager().install()))
  

In [16]:
search_path = '//*[@id="search-input"]'
doi_botton = "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button"
search_botton = "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[3]/button[2]/span[1]"

In [18]:
for count, paper in enumerate(papers):
  if count > 1:
    break
  doi = paper['doi']
  driver.get("https://www.webofscience.com/wos/woscc/basic-search")
  # /html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select
  # Select "DOI" search option
  dropdown = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select'))
  )
  dropdown.click()
  
  # Wait for the "DOI" option to be present and click it
  doi_option = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/div/div[1]/div/div[19]'))
  )
  doi_option.click()

  # Enter the DOI in the search box
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_path))).send_keys(doi)
  
  # Click the search button
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_botton))).click()
  
  # Click the DOI button
  time.sleep(3)
  
  # Click on the first search result
  result = driver.find_element(By.CSS_SELECTOR, 'div.search-results-item')
  result.click()
  
  time.sleep(5)  # Wait for the paper details to load

  # Click on "Full Text at Publisher" link
  full_text_link = driver.find_element(By.LINK_TEXT, 'Full Text at Publisher')
  full_text_link.click()
  
  time.sleep(5) # Wait for the full text to load
  
  body_text = driver.find_element(By.TAG_NAME, 'body').text
  print(body_text)

ElementClickInterceptedException: Message: Element <wos-select> is not clickable at point (384,510) because another element <div class="onetrust-pc-dark-filter ot-fade-in"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:337:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:177:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:136:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:205:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:85:31


In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException

# Initialize the WebDriver for Firefox
driver = webdriver.Firefox()

# Open the website
driver.get('https://www.webofscience.com/wos/woscc/basic-search')

# Define the XPaths

cookie_preferences_xpath = "//*[@id='onetrust-pc-btn-handler']"

confirm_choices_xpath = "//*[@class='save-preference-btn-handler onetrust-close-btn-handler']"

# Confirm cookies option
try:
    # Wait for the "Manage cookie preferences" button to be clickable and click it
    cookie_preferences_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, cookie_preferences_xpath))
    )
    cookie_preferences_button.click()

    # Wait for the "Confirm my choices" button to be clickable and click it
    confirm_choices_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, confirm_choices_xpath))
    )
    confirm_choices_button.click()
    
except TimeoutException:
    print("Timeout while waiting for elements to be clickable.")
except ElementClickInterceptedException:
    print("ElementClickInterceptedException occurred while clicking elements.")

# remove the popup
popup_x = "//*[@id='pendo-close-guide-e0e49982']"
popup = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, popup_x))
)
popup.click()

# Select DOI search option
dropdown_xpath = "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button"

# Wait for the search option box to be clickable and click it
dropdown = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
)

dropdown.click()

doi_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/div/div[1]/div/div[19]"))
)
doi_option.click()



# # Proceed with your original actions
# doi_option_xpath = "//div[@data-id='DI']"
# doi_option = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, doi_option_xpath))
# )
# doi_option.click()

# # Close the WebDriver
# driver.quit()
